# Pivot and Validate Precinct Level Election Results

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np

### Import the precinct level election results


#### [Open Elections - 2018 NY General Election - Precinct Level Results](https://github.com/openelections/openelections-data-al/blob/master/2018/20181106__al__general__precinct.csv)

* Relative Path: `data/precinct_election_results/20181106__al__general__precinct.csv`
* Link: <https://github.com/openelections/openelections-data-al/blob/master/2018/20181106__al__general__precinct.csv>
* Date Accessed: January 31, 2021
* This data appears to be sourced from the official results ([link](https://www.sos.alabama.gov/alabama-votes/voter/election-data), but all the counties have been aggregated into one file and pivoted.
* Of course I will use official county results to ensure that Open Elections didn't make any mistakes.

In [2]:
statewide_election_results_file_path = 'data/precinct_election_results/20181106__al__general__precinct.csv'
statewide_results_df = pd.read_csv(statewide_election_results_file_path,low_memory=False)
statewide_results_df.head(2)

,county,precinct,office,district,party,candidate,votes
0,Autauga,10 JONES COMMUNITY CTR,Attorney General,NaN,DEM,Joseph Siegelman,115
1,Autauga,10 JONES COMMUNITY CTR,Attorney General,NaN,NON,Write-ins,0


The next cell aliases `statewide_results_df` as `df` because typing fewer characters allows for faster data exploration.  

In [3]:
df = statewide_results_df.copy()

### Clean `votes`

In [4]:
def remove_commas(df, col_lst):
    """
    remove commas from the string representation of numbers in the cols in col_lst
    :df: DataFrame object
    :col_lst: list of strings that are each the name of a column :df:
    """
    for col in col_lst:
        if df.dtypes[col] == "object":
            df[col] = df[col].map(lambda s: str(s).replace(",", ""))
        df = df.astype({col: "float"})
        df[col] = df[col].fillna(0)
        df = df.astype({col: "int64"})
    return df

df = remove_commas(df[df.votes.notna()], ['votes'])

### Clean `county`

In [5]:
df.county = df.county.str.strip() + ' County'

In [6]:
df.county.nunique()

67

In [7]:
df.county.unique()

array(['Autauga County', 'Baldwin County', 'Barbour County',
       'Bibb County', 'Blount County', 'Bullock County', 'Butler County',
       'Calhoun County', 'Chambers County', 'Cherokee County',
       'Chilton County', 'Choctaw County', 'Clarke County', 'Clay County',
       'Cleburne County', 'Coffee County', 'Colbert County',
       'Conecuh County', 'Coosa County', 'Covington County',
       'Crenshaw County', 'Cullman County', 'Dale County',
       'Dallas County', 'DeKalb County', 'Elmore County',
       'Escambia County', 'Etowah County', 'Fayette County',
       'Franklin County', 'Geneva County', 'Greene County', 'Hale County',
       'Henry County', 'Houston County', 'Jackson County',
       'Jefferson County', 'Lamar County', 'Lauderdale County',
       'Lawrence County', 'Lee County', 'Limestone County',
       'Lowndes County', 'Macon County', 'Madison County',
       'Marengo County', 'Marion County', 'Marshall County',
       'Mobile County', 'Monroe County', 'Montgom

In [8]:
df.county = df.county.map(lambda county: 'St. Clair County' if county == 'StClair County' else county) 

In [9]:
df.county.unique()

array(['Autauga County', 'Baldwin County', 'Barbour County',
       'Bibb County', 'Blount County', 'Bullock County', 'Butler County',
       'Calhoun County', 'Chambers County', 'Cherokee County',
       'Chilton County', 'Choctaw County', 'Clarke County', 'Clay County',
       'Cleburne County', 'Coffee County', 'Colbert County',
       'Conecuh County', 'Coosa County', 'Covington County',
       'Crenshaw County', 'Cullman County', 'Dale County',
       'Dallas County', 'DeKalb County', 'Elmore County',
       'Escambia County', 'Etowah County', 'Fayette County',
       'Franklin County', 'Geneva County', 'Greene County', 'Hale County',
       'Henry County', 'Houston County', 'Jackson County',
       'Jefferson County', 'Lamar County', 'Lauderdale County',
       'Lawrence County', 'Lee County', 'Limestone County',
       'Lowndes County', 'Macon County', 'Madison County',
       'Marengo County', 'Marion County', 'Marshall County',
       'Mobile County', 'Monroe County', 'Montgom

In [10]:
df.county.nunique()

67

### Clean `office`

In [11]:
offices = list(df.office.unique())
print(len(offices))
print(offices)

12
['Attorney General', 'Ballots Cast', 'Commissioner of Agriculture and Industries', 'Governor', 'Lieutenant Governor', 'Secretary of State', 'State Auditor', 'State House', 'State Senate', 'State Treasurer', 'Straight Party', 'U.S. House']


Ensure that all counties data for each office

In [12]:
county_to_offices = df.groupby('county').office.unique()
county_to_missing_offices = county_to_offices.apply(lambda x: set(offices).difference(x))
county_to_missing_offices[county_to_missing_offices != set()]

Series([], Name: office, dtype: object)

Ensure that all counties data for each office

In [13]:
offices_to_keep = ['Governor', 'Lieutenant Governor', 'State House', 'State Senate', 'U.S. House', 'Straight Party']
df = df[df.office.isin(offices_to_keep)]

Clean `candidate`

In [14]:
df.candidate.value_counts()

Over Votes          14368
Under Votes         14368
Write-ins           11761
Kay Ivey             2240
Walt Maddox          2240
                    ...  
Linda Meigs            13
Thad McClammy          12
Anthony Daniels        12
Mary Wynne Kling       11
Joe Lovvorn            11
Name: candidate, Length: 241, dtype: int64

In [15]:
candidates_to_exclude = ['Under Votes', 'Over Votes', 'Write-ins']
df = df[~df.candidate.isin(candidates_to_exclude)]

Clean `party`

In [16]:
df.party.unique()

array(['DEM', 'REP', 'LIB', 'IND'], dtype=object)

## Validate


#### [Alabama Secretary of State - 2018 AL General Election - County Level Results](https://github.com/openelections/openelections-data-al/blob/master/2018/20181106__al__general__precinct.csv)

* Relative Path: `data/official_county_level_results/2018GeneralVoterTurnoutStats.csv`
* Website Link: <https://www.sos.alabama.gov/alabama-votes/voter/election-data>
* Link: <https://www.sos.alabama.gov/sites/default/files/election-data/2019-01/2018GeneralVoterTurnoutStats.xlsx>
* Date Accessed: January 31, 2021

In [17]:
county_level_election_results_file_path = 'data/official_county_level_results/2018GeneralVoterTurnoutStats.csv'
county_level_election_results_df = pd.read_csv(county_level_election_results_file_path,low_memory=False)
county_level_election_results_df.head(2)

,County,Total Registered Voters,Total Ballots Cast,Straight Party Democrat Ballots Cast,Straight Party Republican Ballots Cast,Total Absentee Ballots Cast,Turnout as a Percentage of Registered Voters,Absentee Ballots Cast as Percentage of Total Ballots Cast
0,Autauga,"\t39,666","19,814","3,666","9,118",725,49.95%,3.66%
1,Baldwin,"\t155,006","78,270","10,465","40,459","2,652",50.49%,3.39%


In [18]:
cdf = county_level_election_results_df.copy()

In [19]:
cdf.County = cdf.County + ' County'
cdf = cdf[cdf.County!='Total County']

In [20]:
cdf = cdf.set_index('County')

In [21]:
cdf.shape

(67, 7)

In [22]:
cdf = remove_commas(cdf, ['Straight Party        Democrat Ballots Cast', 'Straight Party        Republican Ballots Cast'])

In [23]:
(df[(df.office == 'Straight Party') & (df.party == 'DEM')].groupby('county').votes.sum() - cdf['Straight Party        Democrat Ballots Cast']).value_counts()

0    67
dtype: int64

In [24]:
(df[(df.office == 'Straight Party') & (df.party == 'REP')].groupby('county').votes.sum() - cdf['Straight Party        Republican Ballots Cast']).value_counts()

 0    66
-5     1
dtype: int64

Looks like it matches very well. Only off by 5 in one county. It seems like Open Elections did a great job aggregating the files provided by the state.

Pivot the data:

In [25]:
piv = pd.pivot_table(df, index = ['county', 'precinct'], columns = ['office','party'], values = ['votes'], aggfunc = np.sum)
piv.head(2)

votes                              \
office                                Governor         Lieutenant Governor   
party                                      DEM     REP                 DEM   
county         precinct                                                      
Autauga County 10 JONES COMMUNITY CTR    105.0   168.0               109.0   
               100 TRINITY METHODIST     333.0  1436.0               288.0   

                                                                           \
office                                        State House                   
party                                     REP         DEM IND LIB     REP   
county         precinct                                                     
Autauga County 10 JONES COMMUNITY CTR   162.0       127.0 NaN NaN     NaN   
               100 TRINITY METHODIST   1470.0       252.0 NaN NaN  1499.0   

                                                                               \
office                                State Senate             Straight Party   
party                                          DEM IND     REP            DEM   
county         precinct                                                         
Autauga County 10 JONES COMMUNITY CTR          NaN NaN   170.0           98.0   
               100 TRINITY METHODIST           NaN NaN  1494.0          150.0   

                                                                     
office                                           U.S. House          
party                                 LIB    REP        DEM     REP  
county         precinct                                              
Autauga County 10 JONES COMMUNITY CTR NaN  110.0      118.0   153.0  
               100 TRINITY METHODIST  NaN  916.0      391.0  1375.0

In [26]:
piv.columns

MultiIndex([('votes',            'Governor', 'DEM'),
            ('votes',            'Governor', 'REP'),
            ('votes', 'Lieutenant Governor', 'DEM'),
            ('votes', 'Lieutenant Governor', 'REP'),
            ('votes',         'State House', 'DEM'),
            ('votes',         'State House', 'IND'),
            ('votes',         'State House', 'LIB'),
            ('votes',         'State House', 'REP'),
            ('votes',        'State Senate', 'DEM'),
            ('votes',        'State Senate', 'IND'),
            ('votes',        'State Senate', 'REP'),
            ('votes',      'Straight Party', 'DEM'),
            ('votes',      'Straight Party', 'LIB'),
            ('votes',      'Straight Party', 'REP'),
            ('votes',          'U.S. House', 'DEM'),
            ('votes',          'U.S. House', 'REP')],
           names=[None, 'office', 'party'])

In [27]:
office_to_abbreviation = {
    'Governor':'Gov', 
    'Lieutenant Governor':'LtGov',
    'U.S. House':'USHOR',
    'State Senate':'StSen', 
    'State House':'StHOR',
    'Straight Party':'SP',
}

In [28]:
max([len(x) for x in [office_to_abbreviation[office] + '_' + party for _,office,party in piv.columns]])

9

All column names will be under the 10 character limit for shapefiles. Now rename and flatten the columns

In [29]:
piv.columns = [office_to_abbreviation[office] + '_' + party for _,office,party in piv.columns]

#### Save the file

In [30]:
export_df = piv.reset_index()
export_df.head()

,county,precinct,Gov_DEM,Gov_REP,LtGov_DEM,LtGov_REP,StHOR_DEM,StHOR_IND,StHOR_LIB,StHOR_REP,StSen_DEM,StSen_IND,StSen_REP,SP_DEM,SP_LIB,SP_REP,USHOR_DEM,USHOR_REP
0,Autauga County,10 JONES COMMUNITY CTR,105.0,168.0,109.0,162.0,127.0,NaN,NaN,NaN,NaN,NaN,170.0,98.0,NaN,110.0,118.0,153.0
1,Autauga County,100 TRINITY METHODIST,333.0,1436.0,288.0,1470.0,252.0,NaN,NaN,1499.0,NaN,NaN,1494.0,150.0,NaN,916.0,391.0,1375.0
2,Autauga County,110 CENTRAL AL ELECTRIC,149.0,106.0,157.0,97.0,166.0,NaN,NaN,NaN,NaN,NaN,107.0,138.0,NaN,59.0,159.0,94.0
3,Autauga County,140 AUTAUGAVILLE VFD,308.0,280.0,314.0,271.0,351.0,NaN,NaN,NaN,NaN,NaN,296.0,267.0,NaN,186.0,326.0,254.0
4,Autauga County,150 PRATTMONT BAPTIST CH,175.0,472.0,163.0,478.0,158.0,NaN,NaN,482.0,NaN,NaN,507.0,94.0,NaN,300.0,196.0,448.0


In [31]:
export_df.to_csv('data/precinct_election_results/AL_G18_pivoted.csv', index=False)